# Ejercicios Python y Scala

## Capítulo 3

### Ejercicio 2
Leer el CSV del ejemplo del cap2 y obtener la estructura del schema dado por
defecto.

In [0]:
mnm_p = spark.read.csv("/FileStore/tables/mnm_dataset.csv", header=True)

In [0]:
mnm_p.show

Out[11]: <bound method DataFrame.show of DataFrame[State: string, Color: string, Count: string]>

In [0]:
%scala 
val mnm_s = spark.read.option("header",true).csv("/FileStore/tables/mnm_dataset.csv")

mnm_s: org.apache.spark.sql.DataFrame = [State: string, Color: string ... 1 more field]

In [0]:
%scala 
mnm_s.show(5)

+-----+------+-----+
State| Color|Count|
+-----+------+-----+
 TX| Red| 20|
 NV| Blue| 66|
 CO| Blue| 79|
 OR| Blue| 71|
 WA|Yellow| 93|
+-----+------+-----+
only showing top 5 rows

#### Python

In [0]:
mnm_p.schema

Out[12]: StructType(List(StructField(State,StringType,true),StructField(Color,StringType,true),StructField(Count,StringType,true)))

#### Scala

In [0]:
%scala

mnm_s.printSchema()


root
-- State: string (nullable = true)
-- Color: string (nullable = true)
-- Count: string (nullable = true)

### Ejercicio 3
Cuando se define un schema al definir un campo por ejemplo StructField('Delay',
FloatType(), True) ¿qué significa el último parámetro Boolean?

El StructField se utiliza para especificar los nombres de columnas que se quieren poner en es esquema. Luego, el boolean indica si puede hacer nulos en la columna; True indicaría que sí mientras que False lo contrario. Esto indica la siguiente página ```https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.types.StructField.html```

### Ejercicio 4
Dataset vs DataFrame (Scala). ¿En qué se diferencian a nivel de código?

### Ejercicio 5
Utilizando el mismo ejemplo utilizado en el capítulo para guardar en parquet y
guardar los datos en los formatos:

i. JSON

ii. CSV (dándole otro nombre para evitar sobrescribir el fichero origen)

iii. AVRO

#### Python

In [0]:
#JSON
(mnm_p.write.format("json")
 .mode("overwrite")
 .save("/tmp/data/json/mnm_json"))
#CSV
mnm_p.write.format("csv").mode("overwrite").save("/tmp/data/csv/mnm_csv")
#AVRO
(mnm_p.write
 .format("avro")
 .mode("overwrite")
 .save("/tmp/data/avro/mnm_avro"))

#### Scala

In [0]:
%scala
//JSON
mnm_s.write.format("json")
 .mode("overwrite")
 .save("/tmp/data/json/mnm_json_scala")
//CSV
mnm_s.write.format("csv").mode("overwrite").save("/tmp/data/csv/mnm_csv_scala")
//AVRO
mnm_s.write
 .format("avro")
 .mode("overwrite")
 .save("/tmp/data/avro/mnm_avro_scala")

## Ejercicio 6
Revisar al guardar los ficheros (p.e. json, csv, etc) el número de ficheros 
creados, revisar su contenido para comprender (constatar) como se guardan.

i. ¿A qué se debe que hayan más de un fichero?

ii. ¿Cómo obtener el número de particiones de un DataFrame?

iii. ¿Qué formas existen para modificar el número de particiones de un 
DataFrame?

iv. Llevar a cabo el ejemplo modificando el número de particiones a 1 y 
revisar de nuevo el/los ficheros guardados.

i.
Esto es debido a que se guarda el fichero con los datos y varios ficheros que contienen metadatos

In [0]:
#EN CSV
#El fichero
mnm_csv = spark.read.csv("dbfs:/tmp/data/csv/mnm_csv/part-00000-tid-2537299325662574562-f964d42a-a5b5-4cf7-bdfb-12f238154445-21-1-c000.csv", header=True)

In [0]:
mnm_csv.show(5)

+---+------+---+
| TX|   Red| 20|
+---+------+---+
| NV|  Blue| 66|
| CO|  Blue| 79|
| OR|  Blue| 71|
| WA|Yellow| 93|
| WY|  Blue| 16|
+---+------+---+
only showing top 5 rows



In [0]:
%fs
ls '/tmp/data/csv/mnm_csv/'

path,name,size,modificationTime
dbfs:/tmp/data/csv/mnm_csv/_committed_1028874557326275524,_committed_1028874557326275524,212,1651147943000
dbfs:/tmp/data/csv/mnm_csv/_committed_2537299325662574562,_committed_2537299325662574562,200,1651234079000
dbfs:/tmp/data/csv/mnm_csv/_committed_4136834951473676589,_committed_4136834951473676589,113,1651147914000
dbfs:/tmp/data/csv/mnm_csv/_committed_vacuum2926965980024001460,_committed_vacuum2926965980024001460,96,1651234082000
dbfs:/tmp/data/csv/mnm_csv/_started_2537299325662574562,_started_2537299325662574562,0,1651234078000
dbfs:/tmp/data/csv/mnm_csv/part-00000-tid-2537299325662574562-f964d42a-a5b5-4cf7-bdfb-12f238154445-21-1-c000.csv,part-00000-tid-2537299325662574562-f964d42a-a5b5-4cf7-bdfb-12f238154445-21-1-c000.csv,1184854,1651234079000


En la tabla anterior, los 5 primeros ficheros son metadatos los cuales referencian al csv.

ii. Obtener el número de particiones de un DataFrame

Se obtiene con el siguiente código: ````ds.rdd.getNumPartitions```` siendo ds el Dataset o DataFrame

In [0]:
%scala
mnm_s.rdd.getNumPartitions

res17: Int = 1

Hay solo una partición por lo que muestra solo 1.

iii. ¿Qué formas existen para modificar el número de particiones de un DataFrame?

Si quisiésemos más, habría que especificarlo al leer el fichero. De la siguiente forma: ```val mnm_s_par = spark.read.option("header",true).csv("/FileStore/tables/mnm_dataset.csv").repartition(16)```

In [0]:
%scala
val mnm_s_par = spark.read.option("header",true).csv("/FileStore/tables/mnm_dataset.csv").repartition(16)

mnm_s_par: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [State: string, Color: string ... 1 more field]

In [0]:
%scala
mnm_s_par.rdd.getNumPartitions

res22: Int = 16

In [0]:
%scala
mnm_s_par.write.format("csv").mode("overwrite").save("/tmp/data/csv/mnm_csv_scala_16par")

In [0]:
%fs
ls '/tmp/data/csv/mnm_csv_scala_16par'

path,name,size,modificationTime
dbfs:/tmp/data/csv/mnm_csv_scala_16par/_SUCCESS,_SUCCESS,0,1651238888000
dbfs:/tmp/data/csv/mnm_csv_scala_16par/_committed_7619795377295868092,_committed_7619795377295868092,1432,1651238887000
dbfs:/tmp/data/csv/mnm_csv_scala_16par/_started_7619795377295868092,_started_7619795377295868092,0,1651238885000
dbfs:/tmp/data/csv/mnm_csv_scala_16par/part-00000-tid-7619795377295868092-1ec2ae96-ffc1-4259-b05d-618f6c0d2dd2-43-1-c000.csv,part-00000-tid-7619795377295868092-1ec2ae96-ffc1-4259-b05d-618f6c0d2dd2-43-1-c000.csv,74038,1651238886000
dbfs:/tmp/data/csv/mnm_csv_scala_16par/part-00001-tid-7619795377295868092-1ec2ae96-ffc1-4259-b05d-618f6c0d2dd2-44-1-c000.csv,part-00001-tid-7619795377295868092-1ec2ae96-ffc1-4259-b05d-618f6c0d2dd2-44-1-c000.csv,74084,1651238886000
dbfs:/tmp/data/csv/mnm_csv_scala_16par/part-00002-tid-7619795377295868092-1ec2ae96-ffc1-4259-b05d-618f6c0d2dd2-45-1-c000.csv,part-00002-tid-7619795377295868092-1ec2ae96-ffc1-4259-b05d-618f6c0d2dd2-45-1-c000.csv,74052,1651238886000
dbfs:/tmp/data/csv/mnm_csv_scala_16par/part-00003-tid-7619795377295868092-1ec2ae96-ffc1-4259-b05d-618f6c0d2dd2-46-1-c000.csv,part-00003-tid-7619795377295868092-1ec2ae96-ffc1-4259-b05d-618f6c0d2dd2-46-1-c000.csv,74040,1651238886000
dbfs:/tmp/data/csv/mnm_csv_scala_16par/part-00004-tid-7619795377295868092-1ec2ae96-ffc1-4259-b05d-618f6c0d2dd2-47-1-c000.csv,part-00004-tid-7619795377295868092-1ec2ae96-ffc1-4259-b05d-618f6c0d2dd2-47-1-c000.csv,74072,1651238886000
dbfs:/tmp/data/csv/mnm_csv_scala_16par/part-00005-tid-7619795377295868092-1ec2ae96-ffc1-4259-b05d-618f6c0d2dd2-48-1-c000.csv,part-00005-tid-7619795377295868092-1ec2ae96-ffc1-4259-b05d-618f6c0d2dd2-48-1-c000.csv,74026,1651238886000
dbfs:/tmp/data/csv/mnm_csv_scala_16par/part-00006-tid-7619795377295868092-1ec2ae96-ffc1-4259-b05d-618f6c0d2dd2-49-1-c000.csv,part-00006-tid-7619795377295868092-1ec2ae96-ffc1-4259-b05d-618f6c0d2dd2-49-1-c000.csv,74010,1651238885000


Se puede ver que en este caso que están los ficheros de los metadatos y los ficheros con las 16 particiones

## Capítulo 4

### Ejercicio 3
 Leer los AVRO, Parquet, JSON y CSV escritos en el cap3 (Los dos primeros están en los ficheros *Chapter 4_Pyspark* y *Chapter 4_Scala*)

#### Python

In [0]:
#JSON
file = "dbfs:/tmp/data/json/mnm_json/part-00000-tid-6570515074209043752-1c2cd319-6936-47c1-a4ce-f88cb02cb19a-32-1-c000.json"
df_json_p = spark.read.format("json").load(file)
#CSV
df_csv_p = spark.read.csv("dbfs:/tmp/data/csv/mnm_csv/part-00000-tid-5652327037189224389-b98244b7-2bb5-4ec1-98cf-e5477a5b2bdd-33-1-c000.csv", header=True)
#AVRO
df_avro_p = (spark.read.format("avro")
 .load("dbfs:/tmp/data/avro/mnm_avro/part-00000-tid-1804635148172896910-f1b2632a-c937-47de-b56e-601d13ba289a-34-1-c000.avro"))

In [0]:
print(df_json_p.show(5))
print(df_csv_p.show(5))
print(df_avro_p.show(5))

+------+-----+-----+
| Color|Count|State|
+------+-----+-----+
|   Red|   20|   TX|
|  Blue|   66|   NV|
|  Blue|   79|   CO|
|  Blue|   71|   OR|
|Yellow|   93|   WA|
+------+-----+-----+
only showing top 5 rows

None
+---+------+---+
| TX|   Red| 20|
+---+------+---+
| NV|  Blue| 66|
| CO|  Blue| 79|
| OR|  Blue| 71|
| WA|Yellow| 93|
| WY|  Blue| 16|
+---+------+---+
only showing top 5 rows

None
+-----+------+-----+
|State| Color|Count|
+-----+------+-----+
|   TX|   Red|   20|
|   NV|  Blue|   66|
|   CO|  Blue|   79|
|   OR|  Blue|   71|
|   WA|Yellow|   93|
+-----+------+-----+
only showing top 5 rows

None


#### Scala

In [0]:
%scala
//JSON
val df_json_s = spark.read.format("json").load("/tmp/data/json/mnm_json_scala/part-00000-tid-1878550521505857892-d264a3b3-1de3-4c7c-ac6e-98a0a7ae252a-35-1-c000.json")
//CSV
val df_csv_s = spark.read.format("csv")
 .option("header", "true")
 .load("/tmp/data/csv/mnm_csv_scala/part-00000-tid-945854175293185945-b7ee2340-b08f-420e-8f6e-5847d9d8bc89-36-1-c000.csv")
//AVRO
val df_avro_s = spark.read.format("avro")
.load("/tmp/data/avro/mnm_avro_scala/part-00000-tid-3679318734650097166-dc071a35-d528-487a-8c6d-cef9b09b76e1-37-1-c000.avro")

df_json_s: org.apache.spark.sql.DataFrame = [Color: string, Count: string ... 1 more field]
df_csv_s: org.apache.spark.sql.DataFrame = [TX: string, Red: string ... 1 more field]
df_avro_s: org.apache.spark.sql.DataFrame = [State: string, Color: string ... 1 more field]

In [0]:
%scala
df_json_s.show(5)
df_csv_s.show(5)
df_avro_s.show(5)

+------+-----+-----+
 Color|Count|State|
+------+-----+-----+
 Red| 20| TX|
 Blue| 66| NV|
 Blue| 79| CO|
 Blue| 71| OR|
Yellow| 93| WA|
+------+-----+-----+
only showing top 5 rows

+---+------+---+
 TX| Red| 20|
+---+------+---+
 NV| Blue| 66|
 CO| Blue| 79|
 OR| Blue| 71|
 WA|Yellow| 93|
 WY| Blue| 16|
+---+------+---+
only showing top 5 rows

+-----+------+-----+
State| Color|Count|
+-----+------+-----+
 TX| Red| 20|
 NV| Blue| 66|
 CO| Blue| 79|
 OR| Blue| 71|
 WA|Yellow| 93|
+-----+------+-----+
only showing top 5 rows